In [ ]:
import pandas as pd

Reading the Chicago food inspection csv file to dataframe

In [ ]:
df=pd.read_csv("Food_Inspections.csv")

In [ ]:
df

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2522989,IPSENTO,IPSENTO,1741994.0,Restaurant,Risk 2 (Medium),2035 N WESTERN AVE,CHICAGO,IL,60647.0,07/21/2021,Canvass,Out of Business,NaN,41.918575,-87.687296,"(-87.68729592769105, 41.918574636596254)"
1,2522162,FLAT GRILL,FLAT GRILL,2753823.0,Restaurant,Risk 1 (High),2200 W TAYLOR,CHICAGO,IL,60612.0,06/30/2021,Canvass,Out of Business,NaN,41.869148,-87.681526,"(-87.68152576167911, 41.86914817242202)"
2,2521532,7- ELEVEN,7- ELEVEN,1738996.0,Grocery Store,Risk 2 (Medium),600 N MCCLURG CT,CHICAGO,IL,60611.0,06/18/2021,Canvass,Pass,NaN,41.892679,-87.617864,"(-87.61786371164598, 41.89267922007347)"
3,2500345,CHICAGO BY NIGHT,CHICAGO BY NIGHT,1170649.0,Restaurant,Risk 1 (High),5600 W BELMONT AVE,CHICAGO,IL,60634.0,04/16/2021,Non-Inspection,No Entry,NaN,41.938684,-87.766562,"(-87.76656249384537, 41.93868377475747)"
4,2500492,THE NEW GRACE RESTAURANT,THE NEW GRACE RESTAURANT,2712979.0,Restaurant,Risk 1 (High),4409 N BROADWAY,CHICAGO,IL,60640.0,04/06/2021,Non-Inspection,No Entry,NaN,41.962104,-87.655204,"(-87.65520382960334, 41.96210413223745)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224794,176329,P & P AMERICAN & CHINESE FOOD,P & P AMERICAN & CHINESE FOOD,2017568.0,Restaurant,Risk 1 (High),5409 W NORTH AVE,CHICAGO,IL,60639.0,02/23/2010,License Re-Inspection,Pass,NaN,41.909324,-87.760975,"(-87.76097488832582, 41.909323858178425)"
224795,68286,PAPI REAL TACO,PAPI REAL TACO,2016716.0,Restaurant,Risk 1 (High),3544 E 106TH ST,CHICAGO,IL,60617.0,02/02/2010,License,Pass,NaN,41.702852,-87.537139,"(-87.537139292445, 41.70285190603626)"
224796,120312,PINK MONKEY / NEW YORK STRIP,PINK MONKEY / NEW YORK STRIP,52862.0,Restaurant,Risk 1 (High),750 S CLINTON ST,CHICAGO,IL,60607.0,01/25/2010,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.872046,-87.641025,"(-87.64102470530548, 41.87204626658311)"
224797,158267,HARMONY HAMBURGERS,HARMONY HAMBURGERS,2013769.0,Restaurant,Risk 1 (High),3643 E 106TH ST,CHICAGO,IL,60617.0,01/15/2010,License,Fail,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.702622,-87.534808,"(-87.5348083383225, 41.70262227972573)"


Extracting only the restaurants name and the addresses

In [ ]:
df[["DBA Name","Address"]]

,DBA Name,Address
0,IPSENTO,2035 N WESTERN AVE
1,FLAT GRILL,2200 W TAYLOR
2,7- ELEVEN,600 N MCCLURG CT
3,CHICAGO BY NIGHT,5600 W BELMONT AVE
4,THE NEW GRACE RESTAURANT,4409 N BROADWAY
...,...,...
224794,P & P AMERICAN & CHINESE FOOD,5409 W NORTH AVE
224795,PAPI REAL TACO,3544 E 106TH ST
224796,PINK MONKEY / NEW YORK STRIP,750 S CLINTON ST
224797,HARMONY HAMBURGERS,3643 E 106TH ST


Dropping the duplicate restaurants and addresses pairs later to be used for hitting the yelp API

In [ ]:
df_dup_removed=df[["DBA Name","Address"]].drop_duplicates()

In [ ]:
df_dup_removed

,DBA Name,Address
0,IPSENTO,2035 N WESTERN AVE
1,FLAT GRILL,2200 W TAYLOR
2,7- ELEVEN,600 N MCCLURG CT
3,CHICAGO BY NIGHT,5600 W BELMONT AVE
4,THE NEW GRACE RESTAURANT,4409 N BROADWAY
...,...,...
224659,MCDONALD'S,36 W RANDOLPH ST
224712,BACCI PIZZERIA,120 N WELLS ST
224726,NOW WE'RE POPPIN' INC.,111 E WACKER DR
224739,FRESH PANTRY,7337 S SOUTH SHORE DR


Exporting the unique restaurants names and their addresses dataframe to a csv file

In [ ]:
df_dup_removed.to_csv('ChicagoRestaurantsAndAddress.csv', index = False)

In [ ]:
import json
import requests

Making a url string that will store the yelp api and a list of API keys to increase the total number of hits that can be done per day to the yelp API as there is a certain limit of hits per API keys.

In [ ]:
url = 'https://api.yelp.com/v3/businesses/matches'

# List of API keys commented for security reasons

#api_keys = ["####"]

Reading the unique restaurants names and addresses pairs csv to a dataframe and hitting the API for each pairs. For each unique restaurant and addresses pairs 2 API hits are done. First one to extract the business IDs from the yelp api which is stored in the yelp.json file. Second hit is done by using this business ID to get the business details which is stored in yelp.json file.

In [ ]:
rest_df = pd.read_csv('data/ChicagoRestaurantsAndAddress.csv')
rest = rest_df.shape[0]

offset = 0
for i, api_key in enumerate(api_keys):
    headers = {'Authorization': 'Bearer %s' % api_key}
        
    j = 0
    while j in range(2450) and offset <= rest - 1:
        params = {'name': rest_df.iloc[offset]['DBA Name'], 'location': 'Chicago', 'address1': rest_df.iloc[offset]['Address'], 'city': 'Chicago', 'state': 'IL', 'country': 'US', 'match_threshold': 'default'}
        
        req = requests.get(url, params = params, headers = headers)
        data = json.loads(req.text)
        offset += 1
        j += 1

        if(len(data['businesses']) > 0):
            with open("data/yelp.json", "r+") as file:
                data_list = json.load(file)
                data_list.append(data)
                file.seek(0)
                json.dump(data_list, file)

            id_ = data['businesses'][0]['id']
            url_2 = 'https://api.yelp.com/v3/businesses/{}'.format(id_)
            req_2 = requests.get(url_2, headers = headers)
            data_2 = json.loads(req_2.text)

            with open("data/restaurant.json", "r+") as file:
                data_list = json.load(file)
                data_list.append(data_2)
                file.seek(0)
                json.dump(data_list, file)

print(offset)